In [1]:
import numpy as np
import torch
import random

In [2]:
weights = torch.ones(10,10, requires_grad=False)

In [ ]:
torch.tensor(0)

tensor(0)

In [39]:
class Node2Vec(torch.nn.Module):
  def __init__(self, num_nodes : int, embeddings_size : int):
    super(Node2Vec, self).__init__()
    self.num_nodes  = num_nodes
    self.embeddings = torch.nn.Parameter(torch.rand(num_nodes, embeddings_size, requires_grad = True))
    self.softmax = torch.nn.Softmax(dim=0)
  def forward(self, inputGraph : torch.Tensor, neighbours : torch.Tensor):

    if (inputGraph.size(0) == 0 or inputGraph.size(0) != neighbours.size(0)):
      return torch.tensor(np.inf)

    #Performs the loss calculation for the Random Walk algorithm.
    precalculate = self.embeddings.mm(self.embeddings.t())
    probabilities = self.softmax(precalculate)
    rowIndices = torch.arange(neighbours.size(0)).unsqueeze(1).expand_as(neighbours)

    # Combine row indices and values into a tuple-like structure (separate tensors)
    tupleTensor = torch.stack((rowIndices, neighbours), dim=-1)
    tupleList = [tuple(pair.tolist()) for row in tupleTensor for pair in row]
    uniquePairs = set(tupleList)

    loss = torch.tensor(0)
    for i in uniquePairs:
      loss = loss + torch.log(probabilities[int(i[0]), int(i[1])])
    return  -1 * loss




In [16]:
def RandomWalk(inputGraph : torch.Tensor, p : int, q : int, l : int):
    neighbours = torch.zeros(inputGraph.size(0), l, requires_grad=False)
    # Starting at each node
    for i in range(inputGraph.size(0)):
      current = i
      prev = i
      # Filling in the l length sequence
      for j in range(l):
        probs = []
        sum = 0
        rand = random.random()
        print(rand)
        #Checking all neighbours
        for k in range(inputGraph.size(1)):

          # For regular cases.
          if (prev != current):

            # Neighbour connected to previous.
            if (inputGraph[current][k] > 0 and k == current):
              probs.append(inputGraph[current][k] * (1 / q) + sum)
              sum = probs[-1]
            elif (inputGraph[current][k] > 0 and inputGraph[prev][k] > 0 and k != prev):
              probs.append(inputGraph[current][k] + sum)
              sum = probs[-1]
            elif (inputGraph[current][k] > 0 and inputGraph[prev][k] <= 0):
              probs.append(inputGraph[current][k] * (1 / q) + sum)
              sum = probs[-1]
            elif (inputGraph[current][k] > 0 and k == prev):
              probs.append(inputGraph[current][k] * (1 / p) + sum)
              sum = probs[-1]

          # For after the self loop and the start.
          else:
            if (inputGraph[current][k] > 0 and k != current):
              probs.append(inputGraph[current][k] * (1 / q) + sum)
              sum = probs[-1]
            elif (inputGraph[current][k] > 0 ):
              probs.append(inputGraph[current][k] * (1 / p) + sum)
              sum = probs[-1]

        # Calculate the next node and update the variables.
        for index in range(0, len(probs)):
          probs[index] = probs[index] / sum
          if (rand < probs[index]):
            neighbours[i][j] = index
            prev = current
            current = index
            break
    return neighbours



In [40]:
neighbours = RandomWalk(weights, 5, 5, 3)

0.17997185808405447
0.9688311102725632
0.027633039148355687
0.4950636806436235
0.9470544101543421
0.6228254780355968
0.9115942718729367
0.3853167069758856
0.5314928328925437
0.0939068452535261
0.8944113445127033
0.2313465761891097
0.27364622589550225
0.20251473534009623
0.2026570501199394
0.4049555675191069
0.8898480080849143
0.8916642137142667
0.3790557567930011
0.8178119325217175
0.8473518863785658
0.1280308844548328
0.18660204246563206
0.3751164453691277
0.9552592572739552
0.47171960660121426
0.9412689985586399
0.4444462963566991
0.6935353565880515
0.6563973849496896


In [41]:
model = Node2Vec(10,5)
optim = torch.optim.Adam(model.parameters(), lr=1)

In [43]:
for i in range (20000):
  loss = model.forward(weights, neighbours)
  print(loss)
  optim.zero_grad()
  loss.backward()
  optim.step()

Streaming output truncated to the last 5000 lines.
tensor(63.4580, grad_fn=<MulBackward0>)
tensor(63.4711, grad_fn=<MulBackward0>)
tensor(63.4296, grad_fn=<MulBackward0>)
tensor(63.4370, grad_fn=<MulBackward0>)
tensor(63.4091, grad_fn=<MulBackward0>)
tensor(63.4120, grad_fn=<MulBackward0>)
tensor(63.3929, grad_fn=<MulBackward0>)
tensor(63.3906, grad_fn=<MulBackward0>)
tensor(63.3782, grad_fn=<MulBackward0>)
tensor(63.3554, grad_fn=<MulBackward0>)
tensor(63.3500, grad_fn=<MulBackward0>)
tensor(63.3322, grad_fn=<MulBackward0>)
tensor(63.3241, grad_fn=<MulBackward0>)
tensor(63.3195, grad_fn=<MulBackward0>)
tensor(63.3017, grad_fn=<MulBackward0>)
tensor(63.3049, grad_fn=<MulBackward0>)
tensor(63.2786, grad_fn=<MulBackward0>)
tensor(63.2804, grad_fn=<MulBackward0>)
tensor(63.2725, grad_fn=<MulBackward0>)
tensor(63.2596, grad_fn=<MulBackward0>)
tensor(63.2570, grad_fn=<MulBackward0>)
tensor(63.2463, grad_fn=<MulBackward0>)
tensor(63.2433, grad_fn=<MulBackward0>)
tensor(63.2391, grad_fn=<MulB